# Imports 

In [ ]:
from PIL import Image
import numpy as np
from skimage import measure
import time
import gc
from matplotlib import pyplot as plt
from scipy import misc
import openslide as op
import configparser
import glob
import ast
import errno
import os
from util import mkdirs, extend_glob, file_suffix
from evaluate import evaluate

# Configuration

In [ ]:
config = configparser.RawConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read('cytonet.cfg')
section = 'evaluation'

In [ ]:
experiment_folder = config.get(section, 'experiment_folder') if config.has_option(section, 'experiment_folder') \
                    else config.get('general', 'experiment_folder')
predictions = os.path.join(experiment_folder, "prediction","*")
filenames = ast.literal_eval(config.get(section, 'images'))

transparency = config.getint(section, 'transparency') 
display = config.getboolean(section, 'display')
load_level= config.getint(section, 'load_level') if config.has_option(section, 'patch_size') else config.getint('general', 'load_level')

In [ ]:
prediction_folders = glob.glob(predictions)
files = extend_glob(filenames)

In [ ]:
for prediction_folder in prediction_folders:
    print("Working on ", prediction_folder)
    image_name = os.path.split(prediction_folder)[-1]
    image_file = None 
    for file in files:
        if(image_name in file):
            image_file = file
            break
    if(image_file==None):
        raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), image_name)
    predictions_files = glob.glob(os.path.join(prediction_folder,"*"))
    output_folder = os.path.join(experiment_folder,"evaluation", image_name)
    mkdirs(output_folder,0o777)
    for prediction_file in predictions_files:
        evaluate(prediction_file, image_file, output_folder, load_level, transparency, display)